# Python для анализа данных

*Ян Пиле, НИУ ВШЭ*

## Интерактивные визуализации в Plotly v2

In [1]:
# !pip install cufflinks
# !pip install fastparquet
# !pip install pyarrow

Это мой небольшой кавер на:
    
https://nbviewer.jupyter.org/github/WillKoehrsen/Data-Analysis/blob/master/plotly/Plotly%20Whirlwind%20Introduction.ipynb#Introduction:-Plotting-with-Plotly-+-Cufflinks-in-Python

In [2]:
# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [3]:
import os

In [4]:
os.getcwd()+'/medium_data_2019_01_06'

'/Users/Dell/MLDS_python_course_fall22/Untitled Folder/medium_data_2019_01_06'

In [5]:
import pandas as pd
df = pd.read_parquet(os.getcwd()+'/medium_data_2019_01_06')
df.head()

,claps,days_since_publication,fans,link,num_responses,publication,published_date,read_ratio,read_time,reads,...,type,views,word_count,claps_per_word,editing_days,<tag>Education,<tag>Data Science,<tag>Towards Data Science,<tag>Machine Learning,<tag>Python
119,2,574.858594,2,https://medium.com/p/screw-the-environment-but...,0,None,2017-06-10 14:25:00,41.98,7,68,...,published,162,1859,0.001076,0,0,0,0,0,0
118,18,567.540639,3,https://medium.com/p/the-vanquishing-of-war-pl...,0,None,2017-06-17 22:02:00,32.93,14,54,...,published,164,3891,0.004626,0,0,0,0,0,0
121,50,554.920762,19,https://medium.com/p/capstone-project-mercedes...,0,None,2017-06-30 12:55:00,20.19,42,215,...,published,1065,12025,0.004158,0,0,0,0,1,1
122,0,554.078160,0,https://medium.com/p/home-of-the-scared-5af0fe...,0,None,2017-07-01 09:08:00,35.85,9,19,...,published,53,2533,0.000000,0,0,0,0,0,0
114,0,550.090507,0,https://medium.com/p/the-triumph-of-peace-f485...,0,None,2017-07-05 08:51:00,8.77,14,5,...,published,57,3892,0.000000,1,0,0,0,0,0


Типы графиков, которые можно нарисовать с помощью cufflinks:

   
    scatter, bar, box, spread
    ratio, heatmap, surface
    histogram, bubble, bubble3d
    scatter3d, scattergeo, ohlc
    candle, pie, choropleth

**Распределение одной переменной**

Здесь очевидным решением будет гистограмма

In [6]:
df['claps'].iplot(
    kind='hist',
    bins=30,
    xTitle='claps',
    linecolor='black',
    yTitle='count',
    title='Claps Distribution')

На любой прямоугольник можно навести курсор и увидеть значение

**Гистограмма с процентами**

Точно такая же гистограмма в одно действие превращается в распределение с процентами.

In [7]:
df['reads'].iplot(
    kind='hist',
    bins=30,
    xTitle='reads',
    linecolor='black',
    histnorm='percent',
    yTitle='percentage (%)',
    title='Reads Distribution in Percent')

Также на одну гистограмму можно уложить несколько распределений (впрочем, мы это уже проделывали)

In [8]:
def to_time(dt):
    return dt.hour + dt.minute / 60

In [9]:
df['time_started'] = df['started_date'].apply(to_time)
df['time_published'] = df['published_date'].apply(to_time)

df[['time_started', 'time_published']].iplot(
    kind='hist',
    linecolor='black',
    bins=24,
    histnorm='percent',
    bargap=0.1,
    opacity=0.8,
#     barmode='group',
    xTitle='Time of Day',
    yTitle='(%) of Articles',
    title='Time Started and Time Published')

А иногда хочется, чтобы распределении не строились рядом друг с другом, а строились одно поверх другого (с перекрытием). За это отвечает аргумент *barmode* которому надо присвоить значение overlay.

In [10]:
df[['time_published', 'time_started']].iplot(
    kind='hist',
    bins=24,
    linecolor='black',
    opacity=0.6,
    histnorm='percent',
    barmode='overlay',
    xTitle='Time of day',
    yTitle='(%) of articles',
    title='Time Started and Time Published Overlaid')

Столбчатые диаграммы обычно строятся поверх группировки датафрейма по некоторому полю (обычно категориальному). Например давайте посчитаем количество статей в публикации.

In [11]:
'<tag>Education' in df

True

In [12]:
df['publication'].unique()

array(['None', 'Towards Data Science', 'Engineering @ Feature Labs',
       'Noteworthy - The Journal Blog', 'The Reality Project'],
      dtype=object)

In [13]:
df.groupby('publication').count()['fans'].iplot(
    kind='bar', yTitle='Number of Articles', linecolor='black', title='Articles by Publication')

А еще можно строить столбчатые диаграммы по нескольким столбцам (Если в них однородная информация - у нас, например, есть флажки того, что публикация содержит некий тег (например, Python))

In [14]:
[c for c in df if '<tag>' in c]

['<tag>Education',
 '<tag>Data Science',
 '<tag>Towards Data Science',
 '<tag>Machine Learning',
 '<tag>Python']

In [15]:
df[[c for c in df if '<tag>' in c]].sum().iplot(
    kind='bar',
    xTitle='Tag',
    yTitle='Number of Articles with Tag',
    title='Frequency of Tags',
    linecolor='black',
    sortbars=True)

**А еще можно распределение пары признаков рядом нарисовать**

Установим в качестве индекса published_date, приведем дату к месяцу и усредним :).

In [16]:
df2 = df[['views', 'reads',
          'published_date']].set_index('published_date').resample('M').mean()
df2.head()

,views,reads
published_date,,
2017-06-30,463.666667,112.333333
2017-07-31,5521.333333,1207.166667
2017-08-31,6242.800000,993.700000
2017-09-30,2113.000000,279.000000
2017-10-31,NaN,NaN


In [17]:
df3 = df2.dropna()

In [18]:
df3

,views,reads
published_date,,
2017-06-30,463.666667,112.333333
2017-07-31,5521.333333,1207.166667
2017-08-31,6242.800000,993.700000
2017-09-30,2113.000000,279.000000
2017-12-31,60818.750000,15724.000000
2018-01-31,31615.944444,9065.722222
2018-02-28,25463.875000,9933.750000
2018-03-31,69373.500000,18884.000000
2018-04-30,35668.333333,11340.333333


In [19]:
df3.iplot(
    kind='bar',
    xTitle='Дата',
    yTitle='Среднее',
    title='Среднемесячное кол-во просмотров и прочтений')

In [20]:
df2 = df[['views', 'reads',
          'published_date']].set_index('published_date').resample('M').sum()
df3=df2.dropna()

In [21]:
df3['read_percent']= df3['reads']/df3['views']
df3 = df3.dropna()

In [22]:
df3['read_percent'].iplot(
    kind='bar',
    xTitle='Дата',
    yTitle='Среднее',
    title='Доля дочитываний')

Кроме того можно на графике добавить еще одну ось Y

In [23]:
df2 = df[['views', 'read_time',
          'published_date']].set_index('published_date').resample('M').mean()

df2.iplot(
    kind='bar',
    xTitle='Дата',
    secondary_y='read_time',
    opacity=0.4,
    secondary_y_title='Среднее время прочтения',
    yTitle='Среднее кол-во просмотров',
    title='Средние за месяц')

Конечно же Plotly умеет рисовать и ящики с усами - куда же без них.

In [24]:
df[['claps', 'fans']].iplot(secondary_y='fans', secondary_y_title='Fans',
    kind='box', yTitle='Claps', title='BoxPlot Claps и Fans')

Можно нарисовать набор ящиков с усами, например, для нескольких категорий.

In [25]:
df2 = df.pivot(
    columns='publication', values='fans')
df2.head()

publication,Engineering @ Feature Labs,None,Noteworthy - The Journal Blog,The Reality Project,Towards Data Science
0,NaN,NaN,NaN,34.0,NaN
1,NaN,NaN,NaN,29.0,NaN
2,NaN,NaN,NaN,13.0,NaN
3,NaN,34.0,NaN,NaN,NaN
4,NaN,47.0,NaN,NaN,NaN


Всегда в iplot можно явно передать набор параметров в виде словаря(так у нас больше контроля)

In [26]:
df2.iplot(
        kind='box',
        layout=dict(
            height=600,
            yaxis=dict(title='fans'),
            title='Fans by Publication',
            margin=dict(b=140)))

In [27]:
df[df['read_time'] <= 10].pivot(
    columns='read_time', values='reads').iplot(
        kind='box', colorscale='set2',
        xTitle='Время прочтения',
        yTitle='Кол-во прочтений',
        title='Box Plot of Reads by Reading Time')

Сделаем из куска нашего датафрейма временной ряд. Для этого установим дату в индекс

In [28]:
tds = df[df['publication'] == 'Towards Data Science'].set_index(
    'published_date')

tds.head()

,claps,days_since_publication,fans,link,num_responses,publication,read_ratio,read_time,reads,started_date,...,word_count,claps_per_word,editing_days,<tag>Education,<tag>Data Science,<tag>Towards Data Science,<tag>Machine Learning,<tag>Python,time_started,time_published
published_date,,,,,,,,,,,,,,,,,,,,,
2017-12-27 11:20:00,4800,374.986885,861,https://towardsdatascience.com/random-forest-i...,27,Towards Data Science,17.68,21,28566,2017-12-26 15:11:00,...,4494,1.068091,0,0,1,0,1,1,15.183333,11.333333
2018-01-06 20:15:00,857,364.615092,112,https://towardsdatascience.com/improving-rando...,6,Towards Data Science,22.76,17,7207,2018-01-03 21:38:00,...,3504,0.244578,2,0,1,0,1,1,21.633333,20.250000
2018-01-07 20:37:00,186,363.599979,45,https://towardsdatascience.com/data-science-a-...,1,Towards Data Science,28.64,15,775,2018-01-07 13:18:00,...,3569,0.052115,0,0,1,0,0,0,13.300000,20.616667
2018-01-08 16:58:00,119,362.752029,43,https://towardsdatascience.com/a-theory-of-pre...,2,Towards Data Science,31.53,11,740,2018-01-02 17:23:00,...,2817,0.042244,5,0,1,0,0,0,17.383333,16.966667
2018-01-09 21:49:00,2000,361.550093,392,https://towardsdatascience.com/hyperparameter-...,12,Towards Data Science,23.99,12,25505,2018-01-09 12:26:00,...,2456,0.814332,0,0,1,0,1,1,12.433333,21.816667


In [29]:
tds['read_time'].iplot(
    mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Дата',
    yTitle='Минимальное время прочтения',
    title='Read Time Trends')

Конечно же на одном графике можно отображать несколько рядов:

In [30]:
tds[['claps', 'fans']].iplot(
    mode='lines+markers',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='Fans and Claps',
    title='Fans and Claps over Time')

А можно явным образом настроить две оси Y.

In [31]:
tds[['fans', 'word_count', 'title']].iplot(
    y='fans',
    mode='lines+markers',
    secondary_y = 'word_count',
    secondary_y_title='Word Count',
    opacity=0.8,
    size=8,
    symbol=1,
    xTitle='Date',
    yTitle='Claps',
    text='title',
    title='Fans and Word Count over Time')

Данные на графиках можно подписывать, но это делается в html-разметке (На мой взгляд, это единственная не слишком удобная часть plotly)

In [33]:
tds_monthly_totals = tds.resample('M').sum()
tds_monthly_totals

,claps,days_since_publication,fans,num_responses,read_ratio,read_time,reads,title_word_count,views,word_count,claps_per_word,editing_days,<tag>Education,<tag>Data Science,<tag>Towards Data Science,<tag>Machine Learning,<tag>Python,time_started,time_published
published_date,,,,,,,,,,,,,,,,,,,
2017-12-31,4800,374.986885,861,27,17.68,21,28566,4,161596,4494,1.068091,0,0,1,0,1,1,15.183333,11.333333
2018-01-31,26739,4595.367769,5297,117,462.34,130,161342,75,563646,29110,13.738008,15,5,13,6,6,5,203.000000,230.400000
2018-02-28,30346,2299.709918,5533,131,283.11,52,79338,39,203484,12022,26.148524,11,6,6,4,2,2,103.900000,84.083333
2018-03-31,22500,1767.877657,4257,102,176.90,62,113304,50,416241,14064,9.854461,4,5,3,4,1,5,82.833333,69.033333
2018-04-30,18664,1568.069032,3745,58,217.26,52,68042,70,214010,11526,10.667997,29,6,3,4,2,2,93.533333,88.433333
2018-05-31,30000,1154.885482,5859,74,111.96,60,56490,47,266847,14594,9.744425,15,5,5,5,5,1,67.533333,57.833333
2018-06-30,19900,1004.887258,4093,66,112.63,64,58379,41,250806,15224,8.105569,27,5,3,5,5,3,68.250000,57.783333
2018-07-31,3457,521.143492,677,13,75.65,39,8872,26,52230,9263,1.200006,19,3,3,3,2,1,46.083333,29.600000
2018-08-31,14562,1118.462636,2630,56,256.90,81,35385,96,132980,19426,11.008971,53,8,8,7,5,7,115.883333,101.000000


In [34]:
tds_monthly_totals = tds.resample('M').sum()

tds_monthly_totals['text'] = [
    f'<span style="color:blue">{m}<span><br>words: {w:.0f}'
    for m, w in zip(tds_monthly_totals.index.month_name(),
                    tds_monthly_totals['word_count'])
]

tds_monthly_totals.iplot(
    mode='lines+markers+text',
    text='text',
    y='word_count',
    opacity=0.8,
    xTitle='Date',
    yTitle='Word Count',
    title='Total Word Count by Month')

Диаграммы рассеяния мы уже строили, но у них есть еще кое-что, что мы не охватили

In [35]:
tds.iplot(
    x='read_time',
    y='read_ratio',
    xTitle='Read Time',
    yTitle='Reading Percent',
    text='title',
    mode='markers',
    title='Reading Percent vs Reading Time')

In [36]:
tds.sort_values('read_time').iplot(
    x='read_time',
    y='read_ratio',
    xTitle='Read Time',
    yTitle='Reading Percent',
    text='title',
    mode='markers+lines',
    bestfit=True, bestfit_colors=['blue'],
    title='Reading Percent vs Reading Time')

Помимо этого графики могут быть построены "с накоплением"

In [37]:
df.set_index('published_date')[['views', 'word_count']].cumsum().iplot(y='views', secondary_y='word_count',
                                                 yTitle='Views', secondary_y_title='Word Count',
                                                 title='Views and Word Count Totals')

Кроме того, plotly умеет строить парные графики по типу того, что мы делали в seaborn

In [38]:
import plotly.figure_factory as ff

figure = ff.create_scatterplotmatrix(
    df[['claps', 'publication', 'views', 'read_ratio', 'word_count']],
    height=1000,
    width=1000,
    text=df['title'],
    diag='histogram',
    index='publication')
iplot(figure)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



Кроме того можно рисовать график разброса между двумя переменными(Например такое можно провернуть для обменных курсов валют)

In [39]:
df.set_index('published_date')[['views', 'reads']].iplot(
    kind='spread', mode='markers', yTitle='Number', title='Spread between Views and Reads')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/cufflinks/plotlytools.py:850: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.



In [40]:
cufflinks.colors.scales()

Ну и на закуску нарисуем пирожок :)

In [41]:
df.groupby(
    'publication', as_index=False)['reads'].count().iplot(
        kind='pie', labels='publication', values='reads', title='Percentage of Reads by Publication')

И еще один!

In [42]:
df.groupby(
    'publication', as_index=False)['word_count'].sum().iplot(
        kind='pie', labels='publication', values='word_count', title='Percentage of Words by Publication')